In [308]:
# import packages
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
import json

### TRAINING DATASET PREPROCESSING

In [309]:
#import the previously cleaned dataset
train_data_path = r'Processed_datasets/cleaned_train_data.csv'
train_data = pd.read_csv(train_data_path)

In [310]:
train_data.head()

,incidentid,alertid,detectorid,alerttitle,category,incidentgrade,entitytype,evidencerole,filename,folderpath,resourceidname,osfamily,osversion,countrycode,state,city,year,month,day,hour
0,612,123247,7,6,initialaccess,truepositive,ip,related,289573,117668,3586,5,66,31,6,3,2024,6,4,6
1,326,210035,58,43,exfiltration,falsepositive,user,impacted,289573,117668,3586,5,66,242,1445,10630,2024,6,14,3
2,58352,712507,423,298,initialaccess,falsepositive,url,related,289573,117668,3586,5,66,242,1445,10630,2024,6,13,4
3,32992,774301,2,2,commandandcontrol,benignpositive,url,related,289573,117668,3586,5,66,242,1445,10630,2024,6,10,16
4,4359,188041,9,74,execution,truepositive,user,impacted,289573,117668,3586,5,66,242,1445,10630,2024,6,15,1


In [311]:
train_data.nunique()

incidentid         448901
alertid           1248789
detectorid           7813
alerttitle          79952
category               20
incidentgrade           3
entitytype             33
evidencerole            2
filename           222085
folderpath          87832
resourceidname       2283
osfamily                6
osversion              58
countrycode           236
state                1368
city                 9342
year                    2
month                   8
day                    31
hour                   24
dtype: int64

In [312]:
# check for outliers in date manually
unique_year = train_data['year'].unique()
unique_month = train_data['month'].unique()
unique_day = train_data['day'].unique()
unique_hour = train_data['hour'].unique()

print("Unique values in 'year' column:", unique_year)
print("Unique values in 'month' column:", unique_month)
print("Unique values in 'day' column:", unique_day)
print("Unique values in 'hour' column:", unique_hour)

Unique values in 'year' column: [2024 2023]
Unique values in 'month' column: [ 6  5  2  1 12  3  4 11]
Unique values in 'day' column: [ 4 14 13 10 15  6  9  8 12  7 26  3  5 11 29 21 24  1 25 16 28 27 30 31
 23 20  2 22 17 19 18]
Unique values in 'hour' column: [ 6  3  4 16  1 13 23 10  2 18  7  0 21 14 17 20  5  8 22 19 12 11 15  9]


#### outlier treatment

In [313]:
numeric_columns = train_data.select_dtypes(include=['number']).columns

def cap_outliers(df, column, factor=1.5):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - factor * IQR
    upper_bound = Q3 + factor * IQR
    return df[column].clip(lower_bound, upper_bound)

# Apply outlier treatment only to numerical columns
# numerical_columns = train_data.select_dtypes(include=['number']).columns
# for col in numeric_columns:
#     train_data[col] = cap_outliers(train_data, col)

# # Check the result
# print(train_data.nunique())

WE DONT NEED OUTLIER TREATMENT FOR THESE COLUMNS BECAUSE THEY ONLY HAVE DISCRETE IDENTIFIERS AND CATEGORICAL COLUMNS. AS WE MANUALLY CHECKED THE DATETIME WHICH DOESNT CONTAIN ANY OUTLIERS

##### ENCODING

In [314]:
train_data.select_dtypes(include=['object']).columns

Index(['category', 'incidentgrade', 'entitytype', 'evidencerole'], dtype='object')

##### one hot encoding

In [315]:
# One-Hot Encoding
train_data = pd.get_dummies(train_data, columns=['evidencerole'], drop_first=True)

In [316]:
train_data.head(2)

,incidentid,alertid,detectorid,alerttitle,category,incidentgrade,entitytype,filename,folderpath,resourceidname,osfamily,osversion,countrycode,state,city,year,month,day,hour,evidencerole_related
0,612,123247,7,6,initialaccess,truepositive,ip,289573,117668,3586,5,66,31,6,3,2024,6,4,6,True
1,326,210035,58,43,exfiltration,falsepositive,user,289573,117668,3586,5,66,242,1445,10630,2024,6,14,3,False


##### label encoding

In [317]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Identify categorical columns
categorical_columns = ['category', 'incidentgrade', 'entitytype']

# Apply LabelEncoder to each categorical column and save it
for col in categorical_columns:
    # Fit and transform the column using LabelEncoder
    train_data[col] = label_encoder.fit_transform(train_data[col])
    
    # Save the LabelEncoder for this column
    joblib.dump(label_encoder, f'PKL_Files/TRAIN/{col}.pkl')

# Print confirmation message
print("The DataFrame `train_data` now has label encoded categorical columns.")
print("The LabelEncoders for each column are saved as .pkl files in the 'PKL' folder.")

The DataFrame `train_data` now has label encoded categorical columns.
The LabelEncoders for each column are saved as .pkl files in the 'PKL' folder.


In [318]:
train_data.head()

,incidentid,alertid,detectorid,alerttitle,category,incidentgrade,entitytype,filename,folderpath,resourceidname,osfamily,osversion,countrycode,state,city,year,month,day,hour,evidencerole_related
0,612,123247,7,6,10,2,15,289573,117668,3586,5,66,31,6,3,2024,6,4,6,True
1,326,210035,58,43,7,1,32,289573,117668,3586,5,66,242,1445,10630,2024,6,14,3,False
2,58352,712507,423,298,10,1,31,289573,117668,3586,5,66,242,1445,10630,2024,6,13,4,True
3,32992,774301,2,2,1,0,31,289573,117668,3586,5,66,242,1445,10630,2024,6,10,16,True
4,4359,188041,9,74,6,2,32,289573,117668,3586,5,66,242,1445,10630,2024,6,15,1,False


In [319]:
numerical_columns

Index(['incidentid', 'alertid', 'detectorid', 'alerttitle', 'filename',
       'folderpath', 'resourceidname', 'osfamily', 'osversion', 'countrycode',
       'state', 'city', 'year', 'month', 'day', 'hour'],
      dtype='object')

### scaling

In [320]:
# scaler = StandardScaler()

# Select the continuous numerical columns to scale
# columns_to_scale = []

# Apply Standard Scaling
# train_data[columns_to_scale] = scaler.fit_transform(train_data[columns_to_scale])

# Show the scaled data
# print(train_data[columns_to_scale].head())

WE DONT APPLY ANY SCALING BECAUSE WE DONT HAVE ANY CONTINOUS VALUES LIKE PRICE,AGE INSTEAD IT HAS ID,CATEGORICAL COLUMNS

In [321]:
train_data.shape

(9442954, 20)

In [322]:
train_data.describe()

,incidentid,alertid,detectorid,alerttitle,category,incidentgrade,entitytype,filename,folderpath,resourceidname,osfamily,osversion,countrycode,state,city,year,month,day,hour
count,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06
mean,6.956630e+04,4.029340e+05,1.066283e+02,2.778447e+03,8.876213e+00,9.144225e-01,2.040963e+01,2.620834e+05,1.075560e+05,3.583476e+03,4.898994e+00,6.466159e+01,2.236781e+02,1.351642e+03,9.937383e+03,2.024000e+03,5.909347e+00,9.797771e+00,1.215976e+01
std,1.189074e+05,4.566372e+05,4.194168e+02,1.091424e+04,3.820188e+00,8.819000e-01,8.633824e+00,8.177205e+04,3.229548e+04,9.021810e+01,7.019396e-01,9.294453e+00,6.279476e+01,3.507163e+02,2.604671e+03,4.353800e-03,2.906229e-01,6.169538e+00,6.782711e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.023000e+03,1.000000e+00,1.000000e+00,0.000000e+00
25%,5.050000e+02,2.264800e+04,2.000000e+00,2.000000e+00,7.000000e+00,0.000000e+00,1.500000e+01,2.895730e+05,1.176680e+05,3.586000e+03,5.000000e+00,6.600000e+01,2.420000e+02,1.445000e+03,1.063000e+04,2.024000e+03,6.000000e+00,5.000000e+00,6.000000e+00
50%,1.022400e+04,2.135950e+05,9.000000e+00,1.100000e+01,1.000000e+01,1.000000e+00,2.000000e+01,2.895730e+05,1.176680e+05,3.586000e+03,5.000000e+00,6.600000e+01,2.420000e+02,1.445000e+03,1.063000e+04,2.024000e+03,6.000000e+00,9.000000e+00,1.300000e+01
75%,8.304400e+04,6.658840e+05,4.500000e+01,1.720000e+02,1.000000e+01,2.000000e+00,3.100000e+01,2.895730e+05,1.176680e+05,3.586000e+03,5.000000e+00,6.600000e+01,2.420000e+02,1.445000e+03,1.063000e+04,2.024000e+03,6.000000e+00,1.200000e+01,1.800000e+01
max,5.997060e+05,1.721456e+06,9.522000e+03,1.131740e+05,1.900000e+01,2.000000e+00,3.200000e+01,2.895730e+05,1.176680e+05,3.586000e+03,5.000000e+00,6.600000e+01,2.420000e+02,1.445000e+03,1.063000e+04,2.024000e+03,1.200000e+01,3.100000e+01,2.300000e+01


In [323]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9442954 entries, 0 to 9442953
Data columns (total 20 columns):
 #   Column                Dtype
---  ------                -----
 0   incidentid            int64
 1   alertid               int64
 2   detectorid            int64
 3   alerttitle            int64
 4   category              int64
 5   incidentgrade         int64
 6   entitytype            int64
 7   filename              int64
 8   folderpath            int64
 9   resourceidname        int64
 10  osfamily              int64
 11  osversion             int64
 12  countrycode           int64
 13  state                 int64
 14  city                  int64
 15  year                  int64
 16  month                 int64
 17  day                   int64
 18  hour                  int64
 19  evidencerole_related  bool 
dtypes: bool(1), int64(19)
memory usage: 1.3 GB


In [324]:
# Export Datasets
train_data.to_csv(r"Processed_datasets/train_preprocessed.csv", index=False)

### TESTING DATASET PREPROCESSING

In [325]:
#import the previously cleaned dataset
test_data_path = r'Processed_datasets/cleaned_test_data.csv'
test_data = pd.read_csv(test_data_path)

In [326]:
test_data.head()

,incidentid,alertid,detectorid,alerttitle,category,incidentgrade,entitytype,evidencerole,filename,folderpath,resourceidname,osfamily,osversion,countrycode,state,city,year,month,day,hour
0,11767,87199,524,563,lateralmovement,benignpositive,user,impacted,289573,117668,3586,5,66,242,1445,10630,2024,6,4,22
1,91158,632273,2,2,commandandcontrol,benignpositive,machine,impacted,289573,117668,3586,0,0,242,1445,10630,2024,6,3,12
2,32247,131719,2932,10807,lateralmovement,benignpositive,process,related,14,22,3586,5,66,242,1445,10630,2024,6,8,3
3,15294,917686,0,0,initialaccess,falsepositive,cloudlogonsession,related,289573,117668,3586,5,66,242,1445,10630,2024,6,12,12
4,7615,5944,27,18,discovery,benignpositive,user,impacted,289573,117668,3586,5,66,242,1445,10630,2024,6,6,17


In [327]:
test_data.nunique()

incidentid        236267
alertid           637320
detectorid          5970
alerttitle         42265
category              19
incidentgrade          3
entitytype            29
evidencerole           2
filename          122200
folderpath         43970
resourceidname      1109
osfamily               5
osversion             43
countrycode          212
state               1159
city                6637
year                   2
month                  8
day                   31
hour                  24
dtype: int64

In [328]:
# check for outliers in date manually
unique_year = test_data['year'].unique()
unique_month = test_data['month'].unique()
unique_day = test_data['day'].unique()
unique_hour = test_data['hour'].unique()

print("Unique values in 'year' column:", unique_year)
print("Unique values in 'month' column:", unique_month)
print("Unique values in 'day' column:", unique_day)
print("Unique values in 'hour' column:", unique_hour)

Unique values in 'year' column: [2024 2023]
Unique values in 'month' column: [ 6  5  1  2  3  4 12 11]
Unique values in 'day' column: [ 4  3  8 12  6 15 26 25  2  5  7 10 13 31 14 11  9 17 16 23  1 21 27 22
 24 20 30 28 29 18 19]
Unique values in 'hour' column: [22 12  3 17  8  4 11  5 20 21 19  2 16 18 13 23 14  9 10 15  6  7  1  0]


No outlier treatment needed

#### ENCODING

In [329]:
test_data.select_dtypes(include=['object']).columns

Index(['category', 'incidentgrade', 'entitytype', 'evidencerole'], dtype='object')

In [330]:
test_data.head(2)

,incidentid,alertid,detectorid,alerttitle,category,incidentgrade,entitytype,evidencerole,filename,folderpath,resourceidname,osfamily,osversion,countrycode,state,city,year,month,day,hour
0,11767,87199,524,563,lateralmovement,benignpositive,user,impacted,289573,117668,3586,5,66,242,1445,10630,2024,6,4,22
1,91158,632273,2,2,commandandcontrol,benignpositive,machine,impacted,289573,117668,3586,0,0,242,1445,10630,2024,6,3,12


##### label encoding

In [331]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Identify categorical columns
categorical_columns = ['category', 'incidentgrade', 'entitytype']

# Apply LabelEncoder to each categorical column and save it
for col in categorical_columns:
    # Fit and transform the column using LabelEncoder
    test_data[col] = label_encoder.fit_transform(test_data[col])
    
    # Save the LabelEncoder for this column
    joblib.dump(label_encoder, f'PKL_Files/TEST/{col}.pkl')

# Print confirmation message
print("The DataFrame `test_data` now has label encoded categorical columns.")
print("The LabelEncoders for each column are saved as .pkl files in the 'PKL' folder.")

The DataFrame `test_data` now has label encoded categorical columns.
The LabelEncoders for each column are saved as .pkl files in the 'PKL' folder.


In [332]:
test_data.head()

,incidentid,alertid,detectorid,alerttitle,category,incidentgrade,entitytype,evidencerole,filename,folderpath,resourceidname,osfamily,osversion,countrycode,state,city,year,month,day,hour
0,11767,87199,524,563,11,0,28,impacted,289573,117668,3586,5,66,242,1445,10630,2024,6,4,22
1,91158,632273,2,2,1,0,15,impacted,289573,117668,3586,0,0,242,1445,10630,2024,6,3,12
2,32247,131719,2932,10807,11,0,23,related,14,22,3586,5,66,242,1445,10630,2024,6,8,3
3,15294,917686,0,0,10,1,7,related,289573,117668,3586,5,66,242,1445,10630,2024,6,12,12
4,7615,5944,27,18,5,0,28,impacted,289573,117668,3586,5,66,242,1445,10630,2024,6,6,17


In [333]:
train_data.shape

(9442954, 20)

In [334]:
train_data.describe()


,incidentid,alertid,detectorid,alerttitle,category,incidentgrade,entitytype,filename,folderpath,resourceidname,osfamily,osversion,countrycode,state,city,year,month,day,hour
count,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06,9.442954e+06
mean,6.956630e+04,4.029340e+05,1.066283e+02,2.778447e+03,8.876213e+00,9.144225e-01,2.040963e+01,2.620834e+05,1.075560e+05,3.583476e+03,4.898994e+00,6.466159e+01,2.236781e+02,1.351642e+03,9.937383e+03,2.024000e+03,5.909347e+00,9.797771e+00,1.215976e+01
std,1.189074e+05,4.566372e+05,4.194168e+02,1.091424e+04,3.820188e+00,8.819000e-01,8.633824e+00,8.177205e+04,3.229548e+04,9.021810e+01,7.019396e-01,9.294453e+00,6.279476e+01,3.507163e+02,2.604671e+03,4.353800e-03,2.906229e-01,6.169538e+00,6.782711e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.023000e+03,1.000000e+00,1.000000e+00,0.000000e+00
25%,5.050000e+02,2.264800e+04,2.000000e+00,2.000000e+00,7.000000e+00,0.000000e+00,1.500000e+01,2.895730e+05,1.176680e+05,3.586000e+03,5.000000e+00,6.600000e+01,2.420000e+02,1.445000e+03,1.063000e+04,2.024000e+03,6.000000e+00,5.000000e+00,6.000000e+00
50%,1.022400e+04,2.135950e+05,9.000000e+00,1.100000e+01,1.000000e+01,1.000000e+00,2.000000e+01,2.895730e+05,1.176680e+05,3.586000e+03,5.000000e+00,6.600000e+01,2.420000e+02,1.445000e+03,1.063000e+04,2.024000e+03,6.000000e+00,9.000000e+00,1.300000e+01
75%,8.304400e+04,6.658840e+05,4.500000e+01,1.720000e+02,1.000000e+01,2.000000e+00,3.100000e+01,2.895730e+05,1.176680e+05,3.586000e+03,5.000000e+00,6.600000e+01,2.420000e+02,1.445000e+03,1.063000e+04,2.024000e+03,6.000000e+00,1.200000e+01,1.800000e+01
max,5.997060e+05,1.721456e+06,9.522000e+03,1.131740e+05,1.900000e+01,2.000000e+00,3.200000e+01,2.895730e+05,1.176680e+05,3.586000e+03,5.000000e+00,6.600000e+01,2.420000e+02,1.445000e+03,1.063000e+04,2.024000e+03,1.200000e+01,3.100000e+01,2.300000e+01


In [335]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9442954 entries, 0 to 9442953
Data columns (total 20 columns):
 #   Column                Dtype
---  ------                -----
 0   incidentid            int64
 1   alertid               int64
 2   detectorid            int64
 3   alerttitle            int64
 4   category              int64
 5   incidentgrade         int64
 6   entitytype            int64
 7   filename              int64
 8   folderpath            int64
 9   resourceidname        int64
 10  osfamily              int64
 11  osversion             int64
 12  countrycode           int64
 13  state                 int64
 14  city                  int64
 15  year                  int64
 16  month                 int64
 17  day                   int64
 18  hour                  int64
 19  evidencerole_related  bool 
dtypes: bool(1), int64(19)
memory usage: 1.3 GB


In [336]:
# Export Datasets
test_data.to_csv(r"Processed_datasets/test_preprocessed.csv", index=False)